In [33]:
import re
def clean(str):
    p = re.compile('[\(|,|.]')
    s = p.sub('', str)
    return s

In [43]:
# TODO: clean the punc like '(',','.' ... return a set on element in one trial (count?)

treatments = []
one_trial = []
intext = False
with open('data/treatment_description_only.txt_tagged') as file:
    for line in file:
        l = line.split()
        if l == []:
            treatments.append(list(set(one_trial))) # remove duplicate elements
            one_trial = []
        elif len(l)<3:
            pass
        else:
#             print(l)
            tag = l[2]
            if tag=='B-treatment':
                if intext:
                    one_trial.append(clean(intext))
                    intext = False
                intext = l[0]
            elif tag=='I-treatment':
                intext = intext + ' ' + l[0]
        

In [44]:
treatments

[['MOH/IUI treatment'],
 ['intravenous fluids measurement',
  'Fluid resuscitation',
  'microvascular perfusion',
  'IUI Treatment',
  'crystalloid fluid',
  'impaired microvascular perfusion',
  'initial resuscitation',
  'terlipressin therapy',
  'subsequent intra-vascular volume replacement',
  'resuscitation'],
 ['adequate fluid resuscitation']]

In [35]:
clean('(MOH/IUI treatment')

'MOH/IUI treatment'